<a href="https://colab.research.google.com/github/theaidran/ollama_youtube_summarize/blob/main/ollama_youtube_summarize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title #####  Put a YouTube link below and hit Run a cell. Use any-language video and get any-language summary. Quick access to this colab notebook. [tinyurl.com/summarisation](https://tinyurl.com/summarisation) { display-mode: "form" }
%config InteractiveShell.ast_node_interactivity="all" #update python display config

In [9]:
# @title  Summarize { run: "auto", display-mode: "form" }
# All credits to https://github.com/martinopiaggi/summarize
# Good to see links:
# https://github.com/hsiehjackson/RULER
# https://ollama.com/library
# https://amgadhasan.substack.com/p/sota-asr-tooling-long-form-transcription


#1 Choose source url
Link = "https://www.youtube.com/watch?v=t6VoqbAh9HM" #@param {type:"string"}
Type_of_source = "Youtube video or playlist" #@param ['Youtube video or playlist', 'Google Drive video link','Dropbox video link', 'Local file']
Type = Type_of_source
URL = Link
Translate_to = "English" # @param ['Afrikaans', 'Albanian', 'Amharic', 'Arabic', 'Armenian', 'Azerbaijani', 'Basque', 'Belarusian', 'Bengali', 'Bosnian', 'Bulgarian', 'Catalan', 'Cebuano', 'Chinese (Simplified)', 'Chinese (Traditional)', 'Corsican', 'Croatian', 'Czech', 'Danish', 'Dutch', 'English', 'Esperanto', 'Estonian', 'Finnish', 'French', 'Frisian', 'Galician', 'Georgian', 'German', 'Greek', 'Gujarati', 'Haitian Creole', 'Hausa', 'Hawaiian', 'Hebrew', 'Hindi', 'Hmong', 'Hungarian', 'Icelandic', 'Igbo', 'Indonesian', 'Irish', 'Italian', 'Japanese', 'Javanese', 'Kannada', 'Kazakh', 'Khmer', 'Korean', 'Kurdish', 'Kyrgyz', 'Lao', 'Latin', 'Latvian', 'Lithuanian', 'Luxembourgish', 'Macedonian', 'Malagasy', 'Malay', 'Malayalam', 'Maltese', 'Maori', 'Marathi', 'Mongolian', 'Myanmar (Burmese)', 'Nepali', 'Norwegian', 'Nyanja (Chichewa)', 'Pashto', 'Persian', 'Polish', 'Portuguese (Portugal, Brazil)', 'Punjabi', 'Romanian', 'Russian', 'Samoan', 'Scots Gaelic', 'Serbian', 'Sesotho', 'Shona', 'Sindhi', 'Sinhala (Sinhalese)', 'Slovak', 'Slovenian', 'Somali', 'Spanish', 'Sundanese', 'Swahili', 'Swedish', 'Tagalog (Filipino)', 'Tajik', 'Tamil', 'Telugu', 'Thai', 'Turkish', 'Ukrainian', 'Urdu', 'Uzbek', 'Vietnamese', 'Welsh', 'Xhosa', 'Yiddish', 'Yoruba', 'Zulu']

language_codes = {'Afrikaans': 'af', 'Albanian': 'sq', 'Amharic': 'am', 'Arabic': 'ar', 'Armenian': 'hy', 'Azerbaijani': 'az', 'Basque': 'eu', 'Belarusian': 'be', 'Bengali': 'bn', 'Bosnian': 'bs', 'Bulgarian': 'bg', 'Catalan': 'ca', 'Cebuano': 'ceb', 'Chinese (Simplified)': 'zh-CN', 'Chinese (Traditional)': 'zh-TW', 'Corsican': 'co', 'Croatian': 'hr', 'Czech': 'cs', 'Danish': 'da', 'Dutch': 'nl', 'English': 'en', 'Esperanto': 'eo', 'Estonian': 'et', 'Finnish': 'fi', 'French': 'fr', 'Frisian': 'fy', 'Galician': 'gl', 'Georgian': 'ka', 'German': 'de', 'Greek': 'el', 'Gujarati': 'gu', 'Haitian Creole': 'ht', 'Hausa': 'ha', 'Hawaiian': 'haw', 'Hebrew': 'iw', 'Hindi': 'hi', 'Hmong': 'hmn', 'Hungarian': 'hu', 'Icelandic': 'is', 'Igbo': 'ig', 'Indonesian': 'id', 'Irish': 'ga', 'Italian': 'it', 'Japanese': 'ja', 'Javanese': 'jw', 'Kannada': 'kn', 'Kazakh': 'kk', 'Khmer': 'km', 'Korean': 'ko', 'Kurdish': 'ku', 'Kyrgyz': 'ky', 'Lao': 'lo', 'Latin': 'la', 'Latvian': 'lv', 'Lithuanian': 'lt', 'Luxembourgish': 'lb', 'Macedonian': 'mk', 'Malagasy': 'mg', 'Malay': 'ms', 'Malayalam': 'ml', 'Maltese': 'mt', 'Maori': 'mi', 'Marathi': 'mr', 'Mongolian': 'mn', 'Myanmar (Burmese)': 'my', 'Nepali': 'ne', 'Norwegian': 'no', 'Nyanja (Chichewa)': 'ny', 'Pashto': 'ps', 'Persian': 'fa', 'Polish': 'pl', 'Portuguese (Portugal, Brazil)': 'pt', 'Punjabi': 'pa', 'Romanian': 'ro', 'Russian': 'ru', 'Samoan': 'sm', 'Scots Gaelic': 'gd', 'Serbian': 'sr', 'Sesotho': 'st', 'Shona': 'sn', 'Sindhi': 'sd', 'Sinhala (Sinhalese)': 'si', 'Slovak': 'sk', 'Slovenian': 'sl', 'Somali': 'so', 'Spanish': 'es', 'Sundanese': 'su', 'Swahili': 'sw', 'Swedish': 'sv', 'Tagalog (Filipino)': 'tl', 'Tajik': 'tg', 'Tamil': 'ta', 'Telugu': 'te', 'Thai': 'th', 'Turkish': 'tr', 'Ukrainian': 'uk', 'Urdu': 'ur', 'Uzbek': 'uz', 'Vietnamese': 'vi', 'Welsh': 'cy', 'Xhosa': 'xh', 'Yiddish': 'yi', 'Yoruba': 'yo', 'Zulu': 'zu'}

#@markdown ---
#markdown Insert your API key depending on which endpoint you want to use
# API Configuration
api_key = "ollama"
api_endpoint = "Custom"
ollama_model = "llama3" # param ['llama3', 'phi3', 'vicuna:13b-v1.5-16k-q4_0']

endpoints = {
    "Groq": "https://api.groq.com/openai/v1",
    "OpenAI": "https://api.openai.com/v1",
    "Custom": "http://localhost:11434/v1"  # ollama endpoint
}
base_url = endpoints.get(api_endpoint)

models = {
    "Groq": "llama3-8b-8192",
    "OpenAI": "gpt-3.5-turbo",
    "Custom": ollama_model  # Placeholder for any custlom model
}
model = models.get(api_endpoint)

use_Youtube_captions = False #@param {type:"boolean"}
# @markdown  * otherwise transcribe audio


#2 launch ollama server with llama3
try: # to faster next start
  import pytube
except ImportError:
  !curl -fsSL https://ollama.com/install.sh | sh
  !tmux new -d ollama serve
  !ollama run {ollama_model} !


#3 install libs: youtube and whisperx
import subprocess
import re
import os
import IPython
from IPython.display import clear_output

def install_whisperx():
    try:
      import whisperx
    except ImportError:
      print("installing whisperx...may take a few minutes")
      !pip install git+https://github.com/m-bain/whisperx.git > /dev/null
      clear_output()

if use_Youtube_captions:
  !pip install youtube-transcript-api
  from youtube_transcript_api import YouTubeTranscriptApi

if (not Type == "Youtube video or playlist") or (not use_Youtube_captions):
  install_whisperx()

try:
  import deep_translator
except ImportError:
  !pip install -U deep-translator

try:
  import openai
except ImportError:
  !pip install openai
  #clear_output()

import openai
client = openai.OpenAI(api_key=api_key, base_url=base_url)

if Type == "Youtube video or playlist":
  try:
    import pytube
  except ImportError:
    #!pip install git+https://github.com/pytube/pytube
    !pip install git+https://github.com/metalshanked/pytube
    #clear_output()
    from pytube import YouTube

if Type == "Google Drive video link":
  from google.colab import drive
  drive.mount('/gdrive')

#4 Video fetching
skip_transcription=False
text = ""
textTimestamps = ""

def seconds_to_time_format(seconds):
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{int(hours):02d}:{int(minutes):02d}:{int(seconds):02d}"

def download_youtube_audio_only(url):
    yt = YouTube(url)
    audio_stream = yt.streams.get_audio_only()
    saved_path = audio_stream.download(output_path=".", skip_existing=True)
    return saved_path

def check_if_captions_exist(url):
    regex = r'(?:https?:\/\/)?(?:www\.)?(?:youtube\.com\/(?:[^\/\n\s]+\/\S+\/|(?:v|e(?:mbed)?)\/|\S*?[?&]v=)|youtu\.be\/)([a-zA-Z0-9_-]{11})'
    video_id =  re.search(regex, url).group(1)

    try:
      transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
      return (True)
    except:
      return (False)

def download_youtube_captions(url):
    regex = r'(?:https?:\/\/)?(?:www\.)?(?:youtube\.com\/(?:[^\/\n\s]+\/\S+\/|(?:v|e(?:mbed)?)\/|\S*?[?&]v=)|youtu\.be\/)([a-zA-Z0-9_-]{11})'
    video_id =  re.search(regex, url).group(1)
    transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)

    try:
      transcript = YouTubeTranscriptApi.get_transcript(video_id) #languages=['en']
    except:
      for available_transcript in transcript_list:
        if available_transcript.is_translatable:
          transcript = available_transcript.translate('en').fetch()
          break

    text = ""
    for entry in transcript:
            start_time = seconds_to_time_format(entry['start'])
            text += f"{start_time} {entry['text'].strip()}\n"

    transcript_file_name = f"{video_id}_captions.md"

    with open(transcript_file_name, 'w', encoding='utf-8') as f:
      f.write(text)

    return text,transcript_file_name

if Type == "Youtube video or playlist":
    #clean youtube url from timestamp
    URL = re.sub('\&t=\d+s?', '', URL)

    if use_Youtube_captions and check_if_captions_exist(URL):
      text, transcript_file_name = download_youtube_captions(URL)
      skip_transcription=True
    else:
      install_whisperx()
      video_path_local =  download_youtube_audio_only(URL)

elif Type == "Google Drive video link":
  subprocess.run(['ffmpeg', '-y', '-i', '/gdrive/My Drive/' + URL, '-vn', '-acodec', 'pcm_s16le',
                  '-ar', '16000', '-ac', '1', 'gdrive_audio.wav'], check=True)
  video_path_local = "gdrive_audio.wav"

elif Type == "Dropbox video link":
    subprocess.run(['wget', URL, '-O', 'dropbox_video.mp4'], check=True)
    subprocess.run(['ffmpeg', '-y', '-i', 'dropbox_video.mp4', '-vn', '-acodec', 'pcm_s16le',
                    '-ar', '16000', '-ac', '1', 'dropbox_video_audio.wav'], check=True)
    video_path_local = "dropbox_video_audio.wav"

elif Type == "Local file":
    local_file_path = Source
    subprocess.run(['ffmpeg', '-y', '-i', local_file_path, '-vn', '-acodec', 'pcm_s16le',
                    '-ar', '16000', '-ac', '1', 'local_file_audio.wav'], check=True)
    video_path_local = "local_file_audio.wav"


#5 transcription using whisperx
if not skip_transcription:
  import whisperx
  language = "auto" # param {type:"string"}
  #initial_prompt = "" # @param {type:"string"}

  asr_options={"temperatures": 0, "beam_size": 1, "without_timestamps": True,}
  # Run on GPU with FP16
  modelWhisperX = whisperx.load_model("large-v2", device="cuda", compute_type="float16", asr_options=asr_options)

  transcription = modelWhisperX.transcribe(str(video_path_local), batch_size=16,
                                    language=None if language == "auto" else language,
                                    task="translate")

  transcript_file_name = os.path.splitext(video_path_local)[0]+'.md'

  with open(transcript_file_name, 'w') as f:
    for segment in transcription["segments"]:
        start_time = seconds_to_time_format(segment['start'])
        text += f"{start_time} {segment['text'].strip()} "

    f.write(text)


#6 Make a summary
prompt_type = "Summarization"  # param ['Summarization', 'Only grammar correction with highlights']
# markdown Set the number of parallel API calls (be mindful of usage rate limits)
parallel_api_calls = 1 # param


# Define your prompts using a dictionary for easier management
prompts = {
    'Summarization': """Summarize the video transcript excerpt including a concise title that reflects the content. Wrap the title with **markdown bold notation**. Write the summary as if you are continuing a conversation without needing to signal a beginning. Here is the transcript: """,
    'Only grammar correction with highlights': """Repeat the following text correcting any grammatical errors and formatting error. Highlight only the important quote (if there are any) with **markdown bold notation**. Focus solely on the essence of the content as if you are continuing a conversation without using any form of introduction like 'Here's the corrected text:'. Here is the text to fix: """
}

# Select the appropriate prompt
summary_prompt = prompts[prompt_type]



def extract_and_clean_timestamps(text_chunks):
    timestamp_pattern = re.compile(r'(\d{2}:\d{2}:\d{2})')
    cleaned_texts = []
    timestamp_ranges = []
    for chunk in text_chunks:
        timestamps = timestamp_pattern.findall(chunk)
        if timestamps:
            for timestamp in timestamps:
                # Remove each found timestamp from the chunk
                chunk = chunk.replace(timestamp, "")
            timestamp_ranges.append(timestamps[0])  # Assuming you want the first timestamp per chunk
        else:
            timestamp_ranges.append("")
        cleaned_texts.append(chunk.strip())  # Strip to remove any leading/trailing whitespace
    return cleaned_texts, timestamp_ranges

def format_timestamp_link(timestamp):
    if Type == "Youtube video or playlist":
      hours, minutes, seconds = map(int, timestamp.split(':'))
      total_seconds = hours * 3600 + minutes * 60 + seconds
      return f"{timestamp} - {URL}&t={total_seconds}"
    else:
      return f"{timestamp}"

import concurrent.futures
import time
from deep_translator import GoogleTranslator

def summarize(prompt):
    completion = client.chat.completions.create(
            model=model,
            messages=[
            {"role": "system", "content": summary_prompt},
            {"role": "user", "content": prompt}
            ],
            max_tokens=4096
    )
    return completion.choices[0].message.content

def process_and_summarize(text,chunk_size):
    overlap_size = 20
    texts = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size - overlap_size)]
    cleaned_texts, timestamp_ranges = extract_and_clean_timestamps(texts)
    summaries = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=parallel_api_calls) as executor:
        future_to_chunk = {executor.submit(summarize, text_chunk): idx for idx, text_chunk in enumerate(cleaned_texts)}
        for future in concurrent.futures.as_completed(future_to_chunk):
            idx = future_to_chunk[future]
            try:
                summarized_chunk = future.result()
                summary_piece = format_timestamp_link(timestamp_ranges[idx]) + " " + summarized_chunk
                summary_piece += "\n"
                summaries.append((idx, summary_piece))
            except Exception as exc:
                print(f'Chunk {idx} generated an exception: {exc}')
                # Resubmit the task with the new model
                time.sleep(10)
                future_to_chunk[executor.submit(summarize, texts[idx])] = idx

    summaries.sort()  # Ensure summaries are in the correct order
    final_summary =""
    if Translate_to != "English":
      translated_summaries = []
      for _,summary in summaries:
        translated_summary = GoogleTranslator(source='auto', target=language_codes[Translate_to]).translate(summary)
        translated_summaries.append(translated_summary)
        final_summary += translated_summary + "\n\n"
      final_summary = final_summary.strip()
    else :
      final_summary = "\n\n".join([summary for _, summary in summaries])

    # Save the final summary
    final_name = transcript_file_name.replace(".md", "_FINAL_"+str(chunk_size)+".md") if Type != "Dropbox video link" else "final_dropbox_video.md"
    with open(final_name, 'w') as f:
        f.write(final_summary)
    return (final_summary)

from IPython.core.interactiveshell import InteractiveShell # Unblock output
InteractiveShell.ast_node_interactivity = "all"            # It starts to work after restart cell
#%config InteractiveShell.ast_node_interactivity="all"
clear_output()
print ("Generating summary...")
general_summary = process_and_summarize(text,15000) # max string size = max_tokens x 4
IPython.display.Markdown("\n\n **Short Summary:** \n\n" + general_summary)
final_summary = process_and_summarize(text,3072)
clear_output()
IPython.display.Markdown("\n\n **Short Summary:** \n\n" + general_summary + "\n\n" + "\n\n **Long Summary:** \n\n" + final_summary )





 **Short Summary:** 

00:00:00 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=0 **"Witryna internetowa z recenzjami dronów na wyższym poziomie: pisanie atrakcyjnych nagłówków i tworzenie wyróżnionych obrazów"**

W tym fragmencie transkrypcji wideo prelegent omawia, jak napisać skuteczną kopię witryny internetowej i utworzyć wyróżnione obrazy dla witryny z recenzjami dronów. Prelegent rozpoczyna od podania szczegółowych informacji na temat witryny, w tym celu, jakim jest recenzowanie i porównywanie dronów, aby pomóc konsumentom w podejmowaniu świadomych decyzji.

Aby rozpocząć pisanie tekstu strony internetowej, mówca korzysta z podpowiedzi i podaje konkretne przykłady, takie jak utworzenie nagłówka i podtytułu w stylu Alexa Hormozy'ego i Davida Ogilvy'ego. Wygenerowany nagłówek brzmi: „Najlepszy przewodnik dla kupujących drony: Wznieś się na nowy poziom dzięki doskonałemu UAV”, a podtytuł brzmi „Odkryj najlepsze drony odpowiadające Twoim potrzebom i budżetowi: recenzje ekspertów, bezpośrednie porównania i cenne wskazówki dla poufnych osób Podejmij świadomą decyzję.”

Prelegent omawia także narzędzia do generowania obrazów, w tym Mid Journey, Dolly 3, Leonardo AI i Ideogram. Wykorzystują Mid Journey do generowania charakterystycznego obrazu popularnego drona z kamerą lecącego nad panoramą miasta.

Na koniec mówca demonstruje, jak używać serwisu Canva do tworzenia polecanego obrazu w stylu miniatury w serwisie YouTube dla postów na blogu. Zaczynają od stworzenia projektu w serwisie Canva, wybrania opcji „magiczne media” w celu wygenerowania obrazów i wybrania stylu np. filmowego. Wygenerowane obrazy są następnie edytowane w celu usunięcia tła lub dodania nakładek tekstowych.

Ogólnie rzecz biorąc, ten fragment transkrypcji wideo zawiera wskazówki, jak napisać atrakcyjną treść witryny i utworzyć wyróżnione obrazy dla witryny z recenzjami dronów, korzystając z różnych narzędzi i technik w celu zwiększenia atrakcyjności wizualnej witryny.

00:13:01 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=781 **Tworzenie witryn internetowych w oparciu o sztuczną inteligencję: krótka prezentacja**

Film pokazuje, jak zbudować przyzwoitą stronę internetową przy użyciu narzędzi AI. Rozpoczyna się od utworzenia wyróżnionego obrazu w serwisie Canva, co obejmuje wybranie tła, dodanie tekstu i zastosowanie efektów. Następnie prelegent dzieli się swoimi doświadczeniami z różnymi narzędziami opartymi na sztucznej inteligencji używanymi w tym procesie, w tym Reloom do mapowania witryny, Hostinger do tworzenia strony internetowej, ColorMind do wyboru schematu kolorów, FontJoy do sugerowania parowania czcionek oraz ChatGPT lub Clod do pisania kopii. Film pokazuje również wykorzystanie MidJourney, Leonardo, Dolly 3 lub Bing Copilot do generowania obrazów na stronie internetowej. Na koniec podkreśla łatwość tworzenia wyróżnionego obrazu w serwisie Canva i udostępnia zasoby do dalszej eksploracji narzędzi opartych na sztucznej inteligencji.



 **Long Summary:** 

00:00:00 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=0 **Tworzenie witryn internetowych w oparciu o sztuczną inteligencję: uproszczenie procesu**

Kontynuując naszą dyskusję, przyjrzyjmy się fragmentowi transkrypcji wideo, który przedstawia narzędzia obsługujące sztuczną inteligencję do tworzenia stron internetowych bez konieczności posiadania wiedzy technicznej. Prezentowane narzędzie nazywa się Reloom i wykorzystuje sztuczną inteligencję do mapowania całej witryny internetowej. Dzięki dostępnemu planowi bezpłatnemu możliwe jest stworzenie prostej strony internetowej i jeszcze większej liczby planów premium dla zaawansowanych użytkowników. Wprowadzając monit dotyczący tematu witryny, Reloom generuje mapę witryny z różnymi sekcjami, w tym stronami takimi jak strona główna, recenzje, ceny i często zadawane pytania. Narzędzie może także generować treść dla każdej strony, dzięki czemu użytkownicy mogą stworzyć kompleksowy projekt strony internetowej bez konieczności zaczynania od zera.

00:02:46 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=166 **Tworzenie witryny internetowej opartej na sztucznej inteligencji w kilka minut**

Fragment transkrypcji wideo omawia, w jaki sposób sztuczna inteligencja może zbudować stronę internetową w ciągu kilku minut, korzystając z platformy WordPress. Prelegent wybiera plan tworzenia witryn biznesowych, który obejmuje funkcje AI, i zawiera umowę, wprowadzając kod kuponu „Matt Wolf” bez spacji. Następnie demonstrowany jest proces tworzenia witryny internetowej, a sztuczna inteligencja konfiguruje witrynę na podstawie dostarczonego opisu. Prelegent podkreśla, że ​​strona internetowa już działa i można ją edytować w razie potrzeby.

Streszczenie:

00:05:46 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=346 **Tworzenie strony internetowej za pomocą sztucznej inteligencji: schemat kolorów i czcionki**

Kontynuując od miejsca, w którym skończyliśmy, pomyślnie utworzyliśmy paletę kolorów przy użyciu narzędzi AI, takich jak Color Mind, zmieniając kolory na nasz nowy wybór. Następnie użyliśmy Font Joy, aby znaleźć parę czcionek, która odpowiada naszym gustom, decydując się na skróconą Roboto jako naszą główną czcionkę i Work Sans dla podnagłówków.

Teraz, gdy mamy już określone podstawy naszej witryny, w tym jej szkielety oparte na Hostingerze, schemat kolorów i pary czcionek, czas zacząć uzupełniać zawartość. Będziemy używać narzędzi AI, takich jak Chat GPT lub Claude, aby wygenerować tekst odpowiadający naszym potrzebom. Aby uzyskać pożądany styl wyjściowy, musimy stworzyć przemyślaną zachętę. W tym przypadku tworzymy stronę internetową z recenzjami sprzętu do dronów, a naszym przykładem będzie Claude.

00:08:11 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=491 **Najlepszy przewodnik dla kupujących drony: Wzlot na nowe wyżyny**

Odkryj drony najlepsze dla Twoich potrzeb i budżetu: recenzje ekspertów, bezpośrednie porównania i cenne wskazówki, które pomogą Ci podjąć świadomą decyzję.

To podsumowanie dotyczy stworzenia strony internetowej zawierającej recenzje i porównania dronów, aby pomóc konsumentom w podejmowaniu świadomych decyzji przy zakupie drona. Celem jest napisanie angażującego tekstu witryny, zawierającego nagłówek i podtytuł w stylu Alexa Hormozy'ego i Davida Ogilvy'ego. Fragment transkrypcji wideo opisuje generowanie obrazów za pomocą popularnego generatora obrazów Mid Journey do wykorzystania na stronie internetowej.

00:11:00 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=660 **Podnieś swoją obecność w Internecie dzięki treściom wizualnym**

Prelegent dzieli się swoim procesem tworzenia wysokiej jakości treści wizualnych przy użyciu funkcji generatora obrazów Canva. Zaczynają od zastąpienia przestarzałego obrazu na swojej stronie internetowej nowym, wyśrodkowania go i dostosowania jego wyrównania za pomocą funkcji siatki Canva. Następnie tworzą wyróżniony obraz w stylu miniatury YouTube dla postów na blogu, korzystając z wbudowanych opcji szablonów Canva i funkcji „Magic Media”, która generuje obrazy o spójnych stylach. Eksperymentują z różnymi szablonami i generują wiele obrazów do wyboru. Prelegent podkreśla znaczenie atrakcyjnej wizualnie treści, która przyciąga kliknięcia i zachęca użytkowników do zaangażowania się w swoją obecność w Internecie.

00:13:01 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=781 **Tworzenie strony internetowej za pomocą sztucznej inteligencji: krótki przewodnik**

Prelegent nadal opisuje, w jaki sposób wykorzystali różne narzędzia sztucznej inteligencji do szybkiego zbudowania witryny internetowej, w tym Canva do wyświetlania polecanych obrazów, Reloom do mapy witryny, Hostinger z kodem kuponu MattWolf do hostingu i nie tylko. Podkreślają, jak te narzędzia mogą pomóc w stworzeniu „całkiem przyzwoitej” witryny internetowej w krótkim czasie i przy minimalnym wysiłku. Prelegent twierdzi również, że chociaż sztuczna inteligencja nie jest jeszcze doskonała, jest już bliska stworzenia dopracowanej strony internetowej, która wygląda jak wielomilionowa firma.

00:16:08 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=968 **Sponsorowanie i prezenty: budowanie społeczności**

Rozmowa jest kontynuowana, a gospodarz wyraża wdzięczność Hostingerowi za sponsorowanie filmu i zapowiedź przyszłych rozdawania narzędzi i gadżetów AI. Gospodarz zaprasza widzów do dołączenia do bezpłatnego biuletynu na futuretools.io, gdzie mogą otrzymywać aktualizacje na temat ciekawych narzędzi AI i nowości. Aby wziąć udział w rozdaniu na kamerę Insta360 X4 należy zapisać się zarówno na kanał YouTube, jak i do newslettera. Celem gospodarza jest zbudowanie społeczności osób o podobnych poglądach poprzez udostępnianie subskrybentom przydatnych narzędzi i gadżetów.